In [1]:
import json
import os
import pandas as pd
import datetime as dt
import folium


path = os.listdir(path="C:/Users/sagal/Desktop/Studia/1SEM/INTOBL/rowery")

files = []

#rozdzielanie nazwy pliku na datę - rok miesiac dzien godzina
for name in path:
    time = dt.datetime(int(name[0:4]), int(name[4:6]), int(name[6:8]), int(name[9:11]), int(name[11:13]), int(name[13:15]))
    files.append((time, name))

files = sorted(files)

In [2]:
def func():
    stat_ind = []
    global df
    for file in files:
        with open('C:/Users/sagal/Desktop/Studia/1SEM/INTOBL/rowery/'+ file[1]) as bicycles:
            data = json.load(bicycles)
            stations = data[0]["places"]
            df = pd.DataFrame(stations)
            
        return df
   # for x in range(len(df)-2):
      #  stat_ind.append(x)
   # print(stat_ind[:])    #tablica stat_ind służyła mi tylko do sprawdzenia poprawności otrzymanych danych (ilość i numery indeksów)

index = func()
        
#print(len(df)-5)   #sprawdzam jaką liczbę zwróci mi program, by być pewna, że zgadza się z ilością indeksów - stacji

In [3]:
#przygotowanie danych do porównania
def bikes_at_station(files):
    bikes_list = []
    bikes = []
    for file in files:
        with open('C:/Users/sagal/Desktop/Studia/1SEM/INTOBL/rowery/' + file[1]) as jsons:
            data = json.load(jsons)
            if data == []:
                continue
            for ind in range(len(df)-5):
                bik = data[0]['places'][ind]["bike_numbers"]
                if bik == None: 
                    bik = "" #brak rowerów na stacji
                bikes.append([ind, bik])
                
            bikes_list.append(bikes)
            bikes = []
            
    return bikes_list 



In [4]:
bt = bikes_at_station(files)
print(len(bt))

4177


In [ ]:



#rozpoczynamy badania aktywności na stacjach - wypożyczone rowery oraz oddane na stacje
activity = []
taken_from_stations = []
given_to_stations = []

count =0
counter = 0
w = 0

for w in range(len(df)-5):
    count =0
    counter = 0
    n = 0
    station_count = 0
    stat_count2 = 0
    tab = []
    
    for nn in range(len(bt)-1):
        bike_list = bt[n][w]   
        tab.append([n,bike_list[0],bike_list[1]])
        n+=1
       
    #Zliczanie wypożyczonych rowerów na stacji 'w'
       
    for mn in range(len(bt)-2):
        bikes_then = tab[mn][2].split(",")
        bikes_now = tab[mn+1][2].split(",")
        
        for c in bikes_then:
            if c not in bikes_now:
                count += 1
               
        station_count += count
        count = 0
    taken_from_stations.append([w, station_count])
    

 

    #Zliczanie oddanych rowerów na stacji 'w'
    
    for i in range(1, len(bt)-1):
        bike_numbers = tab[i][2].split(",")
        
        for b in bike_numbers:
            if b not in tab[i - 1][2]:
                counter += 1
                
                
        stat_count2 += counter
        counter = 0
    given_to_stations.append([w,stat_count2])
    #print(given_to_stations[:])

    activity.append([w, given_to_stations[w][1]+taken_from_stations[w][1]])


#print(activity)
#print(taken_from_stations)

In [ ]:
#Sprawdzanie największej i najmniejszej aktywności na stacjach, a także dla informacji dodatkowej wypożyczeń i zwrotów
def maks(function):
    k = 0
    for l in range(len(df)-5):
        r = function[l][1]
        if r > k:        
             k = r
     
    print('największa ilość:',k)

def min(function):
    k = 0
    for l in range(len(df)-5):
        r = function[l][1]
        if r < k:        
            k = r
     
    print('najmniejsza ilość:',k)
    
    
print('aktywność:')  
maks(activity)
min(activity)

print('\nwypożyczenia:')
maks(taken_from_stations)
min(taken_from_stations)

print('\nzwrot rowerów na stację:')
maks(given_to_stations)
min(given_to_stations)

In [ ]:
m = folium.Map(location=[52.2298,21.0118], zoom_start=10)
from folium.plugins import MarkerCluster

#Marker cluster został zakomentowany, ponieważ w przypadku aktywności na stacjach grupowanie punktów wpływa negatywnie na ich czytelność
#marker_cluster = MarkerCluster().add_to(m)   

for i in range(0,len(df)-5):
      f = str(i)
      p = str(activity[i][1])  
    
      if activity[i][1] > 3000:  
        folium.Marker(
          location=[df.iloc[i]['lat'], df.iloc[i]['lng']],
          popup='index: ' + f + '\n' + df.iloc[i]['name'] + '\naktywność: ' + p,
          icon = folium.Icon(color="red",icon="angle-double-up", prefix='fa')
          ).add_to(m)
      if activity[i][1] in range(1800,3000):
        folium.Marker(
          location=[df.iloc[i]['lat'], df.iloc[i]['lng']],
          popup='index: ' + f + '\n' + df.iloc[i]['name'] + '\naktywność: ' + p,
          icon = folium.Icon(color="orange",icon="angle-up", prefix='fa')
          ).add_to(m)
      if activity[i][1] in range(900,1800):
        folium.Marker(
          location=[df.iloc[i]['lat'], df.iloc[i]['lng']],
          popup='index: ' + f + '\n' + df.iloc[i]['name'] + '\naktywność: ' + p,
          icon = folium.Icon(color="blue",icon="angle-down", prefix='fa')
          ).add_to(m)
      if activity[i][1] in range(0,900):
        folium.Marker(
          location=[df.iloc[i]['lat'], df.iloc[i]['lng']],
          popup='index: ' + f + '\n' + df.iloc[i]['name'] + '\naktywność: ' + p,
          icon = folium.Icon(color="darkblue",icon="angle-double-down", prefix='fa')
          ).add_to(m)
m

In [ ]:
print(given_to_stations[329])